In [3]:
import pandas as pd
import re

DISTRICT_ALL_PATH = "../district_all.csv"  # 업로드된 district_all.csv 경로

# ----------------------------
# 1) district_all 로드 + 기본 전처리
# ----------------------------
district = pd.read_csv(DISTRICT_ALL_PATH, encoding="cp949")
district = district.copy()

district["district_code"] = district["district_code"].astype(str).str.strip()
district["district_name"] = district["district_name"].astype(str).str.strip()
district["available"] = district["available"].astype(str).str.strip()

# 존재만 사용(원하면 '가능' 등도 포함하도록 수정)
district_alive = district[district["available"].isin(["존재", "가능", "Y", "1"])].copy()

# ----------------------------
# 2) 시도(최상위) 룩업: "서울특별시" 같은 한 토큰(공백 없는) 이름
#    - 법정동 코드에서 시도 레벨은 뒤 8자리가 0인 경우가 많음 (예: 1100000000)
#    - 여기서는 '공백 없는 district_name'을 시도로 본다(가장 안전)
# ----------------------------
sido_rows = district_alive[district_alive["district_name"].str.contains(" ") == False].copy()
sido_name_to_code = dict(zip(sido_rows["district_name"], sido_rows["district_code"]))

def normalize_text(x: str) -> str:
    if x is None:
        return ""
    x = str(x).strip()
    x = re.sub(r"\s+", " ", x)
    return x

def expand_sido_to_official(sido_raw: str) -> str | None:
    """
    입력 sido가 '서울'처럼 축약일 수 있으니 district_all의 공식 명칭으로 최대한 맞춘다.
    """
    s = normalize_text(sido_raw)
    if not s:
        return None

    # 1) 이미 공식명이면 그대로
    if s in sido_name_to_code:
        return s

    # 2) 흔한 접미 후보들
    candidates = [
        s,
        s + "특별시",
        s + "광역시",
        s + "특별자치시",
        s + "도",
        s + "특별자치도",
    ]
    for c in candidates:
        if c in sido_name_to_code:
            return c

    # 3) 그래도 없으면: district_all 시도명 중 "시작 문자열"로 유일 매칭 시도
    hits = [name for name in sido_name_to_code.keys() if name.startswith(s)]
    if len(hits) == 1:
        return hits[0]

    return None  # 못 찾음 / 모호

# ----------------------------
# 3) 시군구 룩업 생성
#    - district_name에서 "시도 " 이후를 tail로 보고,
#      tail의 앞부분을 시군구 후보로 생성(예: "수원시 장안구", "강남구")
# ----------------------------
def build_sigungu_candidates_for_row(district_name: str) -> list[str]:
    """
    district_name = "경기도 수원시 장안구 영화동" 같은 형태에서
    시군구 후보를 ["수원시", "수원시 장안구"]처럼 뽑아낸다.
    """
    tokens = normalize_text(district_name).split(" ")
    if len(tokens) < 2:
        return []

    # tokens[0] = 시도
    tail = tokens[1:]

    cands = []
    # 1토큰 시군구(예: "강남구", "춘천시", "홍천군")
    if tail[0].endswith(("시", "군", "구")):
        cands.append(tail[0])

    # 2토큰 시군구(예: "수원시 장안구", "청주시 흥덕구")
    if len(tail) >= 2:
        if tail[0].endswith("시") and tail[1].endswith("구"):
            cands.append(tail[0] + " " + tail[1])

    return list(dict.fromkeys(cands))  # 중복 제거 + 순서 유지

# sido_std -> { sigungu_std -> code }
sigungu_map = {}  # (sido_std, sigungu_std) -> code
sigungu_prefix_index = {}  # (sido_std) -> list of (sigungu_std, code)

for _, r in district_alive.iterrows():
    name = r["district_name"]
    code = r["district_code"]

    tokens = normalize_text(name).split(" ")
    if len(tokens) < 2:
        continue
    sido_std = tokens[0]

    for sg in build_sigungu_candidates_for_row(name):
        key = (sido_std, sg)
        # 같은 키가 여러 번 나오면 "가장 상위(짧은 명칭)"에 가까운 코드를 쓰기 위해
        # 우선은 '최소 코드'로 고정(일반적으로 상위일수록 작은 코드인 경향이 있음)
        if key not in sigungu_map:
            sigungu_map[key] = code
        else:
            sigungu_map[key] = min(sigungu_map[key], code)

# index (빠른 탐색)
for (sido_std, sg), code in sigungu_map.items():
    sigungu_prefix_index.setdefault(sido_std, []).append((sg, code))

# ----------------------------
# 4) 매핑 함수: 입력 df의 sido/sigungu를 코드로 정리
# ----------------------------
def map_region_codes(df: pd.DataFrame, sido_col="sido", sigungu_col="sigungu") -> pd.DataFrame:
    out = df.copy()
    out[sido_col] = out[sido_col].map(normalize_text)
    out[sigungu_col] = out[sigungu_col].map(normalize_text)

    # sido 공식화 + 코드
    out["sido_std"] = out[sido_col].apply(expand_sido_to_official)
    out["sido_code"] = out["sido_std"].map(sido_name_to_code)

    # sigungu 매칭(3단계: 정확 -> 포함/부분 -> 실패/모호)
    sigungu_std_list = []
    sigungu_code_list = []
    match_status_list = []

    for _, row in out.iterrows():
        sido_std = row["sido_std"]
        sg_raw = row[sigungu_col]

        if not sido_std or not sg_raw:
            sigungu_std_list.append(None)
            sigungu_code_list.append(None)
            match_status_list.append("FAIL:missing")
            continue

        # A) exact match: (sido_std, sg_raw)
        key = (sido_std, sg_raw)
        if key in sigungu_map:
            sigungu_std_list.append(sg_raw)
            sigungu_code_list.append(sigungu_map[key])
            match_status_list.append("OK:exact")
            continue

        # B) candidates within this sido
        candidates = sigungu_prefix_index.get(sido_std, [])

        # B1) substring 포함(예: sg_raw="장안구" -> "수원시 장안구" 후보)
        hits = [(sg, code) for (sg, code) in candidates if sg_raw in sg]
        if len(hits) == 1:
            sg, code = hits[0]
            sigungu_std_list.append(sg)
            sigungu_code_list.append(code)
            match_status_list.append("OK:contains")
            continue
        elif len(hits) > 1:
            # 더 짧은 이름 우선(구체/상위 충돌 방지)
            hits_sorted = sorted(hits, key=lambda x: len(x[0]))
            sg, code = hits_sorted[0]
            sigungu_std_list.append(sg)
            sigungu_code_list.append(code)
            match_status_list.append(f"AMBIG:contains({len(hits)})")
            continue

        # B2) startswith(예: sg_raw="수원시" -> "수원시 장안구"는 아니지만 "수원시" 자체 후보가 있으면)
        hits = [(sg, code) for (sg, code) in candidates if sg.startswith(sg_raw) or sg_raw.startswith(sg)]
        if len(hits) == 1:
            sg, code = hits[0]
            sigungu_std_list.append(sg)
            sigungu_code_list.append(code)
            match_status_list.append("OK:prefix")
            continue
        elif len(hits) > 1:
            hits_sorted = sorted(hits, key=lambda x: len(x[0]))
            sg, code = hits_sorted[0]
            sigungu_std_list.append(sg)
            sigungu_code_list.append(code)
            match_status_list.append(f"AMBIG:prefix({len(hits)})")
            continue

        # C) fail
        sigungu_std_list.append(None)
        sigungu_code_list.append(None)
        match_status_list.append("FAIL:notfound")

    out["sigungu_std"] = sigungu_std_list
    out["sigungu_code"] = sigungu_code_list
    out["match_status"] = match_status_list
    return out

# ----------------------------
# 5) 사용 예시
# ----------------------------
# 예: crosswalks_passenger_unique.csv를 불러와 정리 후 저장
# df_cw = pd.read_csv("/mnt/data/crosswalks_passenger_unique.csv", encoding="utf-8-sig")
# df_cw2 = map_region_codes(df_cw, sido_col="sido", sigungu_col="sigungu")
# df_cw2.to_csv("/mnt/data/crosswalks_passenger_unique_with_codes.csv", index=False, encoding="utf-8-sig")

# 예: signals_passenger_unique.csv도 동일
# df_sg = pd.read_csv("/mnt/data/signals_passenger_unique.csv", encoding="utf-8-sig")
# df_sg2 = map_region_codes(df_sg, sido_col="sido", sigungu_col="sigungu")
# df_sg2.to_csv("/mnt/data/signals_passenger_unique_with_codes.csv", index=False, encoding="utf-8-sig")

# 실패/모호 케이스만 따로 뽑기(수동 교정용)
# bad = df_cw2[df_cw2["match_status"].str.startswith(("FAIL", "AMBIG"))]
# bad.to_csv("/mnt/data/cw_region_mapping_issues.csv", index=False, encoding="utf-8-sig")


In [4]:
df_cw = pd.read_csv("../crosswalks_passenger_unique.csv", encoding="utf-8-sig")
df_cw2 = map_region_codes(df_cw, sido_col="sido", sigungu_col="sigungu")
df_cw2.to_csv("../cw.csv", index=False, encoding="utf-8-sig")

In [ ]:
df_sg = pd.read_csv("../signals_passenger_unique.csv", encoding="utf-8-sig")
df_sg2 = map_region_codes(df_sg, sido_col="sido", sigungu_col="sigungu")
df_sg2.to_csv("../sg.csv", index=False, encoding="utf-8-sig")

C:\Users\user\AppData\Local\Temp\ipykernel_23980\3679489485.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sg = pd.read_csv("../signals_passenger_unique.csv", encoding="utf-8-sig")


In [6]:
df_acc = pd.read_csv("../sido_sigungu_month_acc_wide_with_district.csv", encoding="utf-8-sig")
df_acc2 = map_region_codes(df_acc, sido_col="sido", sigungu_col="sigungu")
df_acc2.to_csv("../acc.csv", index=False, encoding="utf-8-sig")